###**Описание задачи:**
Восстановить пропущенные пробелы в текстах объявлений, где слова слиты в одну строку.

###**Предлагаемое решение**
Гибридный подход, комбинирующий сильные стороны разных методов через взвешенное голосование. Каждый метод вносит свой вклад в окончательное решение, что позволяет достичь максимальной точности.

###**Почему этот подход эффективен?**
Гибридный подход эффективен благодаря совмещению четырех методов:
*   Словарный поиск - Быстрое покрытие известных слов

*   Алгоритм Витерби - Глобальная оптимизация разбиения

*   Распознавание паттернов - Точечное решение специфичных случаев

*   Лингвистические эвристики - Вспомогательная проверка

###**Словарь**

Словарь специально подобран для объявлений Avito и включает:

*   Действия: куплю, продам, сдам, ищу, сниму

*   Недвижимость: квартира, дом, комната, офис, гараж

*   Техника: айфон, телефон, ноутбук, телевизор

*   Мебель: диван, кровать, стол, стул, шкаф

*   Транспорт: машина, авто, велосипед

*   Прилагательные: новый, б/у, срочно, недорого

*   Локации: москва, питер, метро, центр




###**Анализ алгоритма:**

Суть алгоритма:

*   Параллелизм: Четыре независимых метода анализируют текст одновременно

*   Взвешенное решение: Каждый метод вносит вклад согласно своей надежности

*   Интеллектуальная фильтрация: Удаление неправдоподобных разбиений по лингвистическим правилам

Преимущества:
*   Компенсация слабостей: Каждый метод покрывает недостатки других
*   Доменная оптимизация: Специализация под лексику Avito
*   Воспроизводимость: Детерминированные алгоритмы, прозрачная логика

Ключевое значение здесь играет качетво словаря, спецификация под домен Авито и его объем (от него зависит и словарный поиск и алгоритм Витерби, который может подстроиться под "шум" словаря). Пример словаря я попросил сгенерировать LLM (в силу ограниченности времени), что не способствовало высокому показателю метрики.
Распознавание же паттернов тоже весьма ограничено - требует ручной прописи всех возможных паттернов.
Однако, в целом данный подход имеет значительное преимущество в скорости в сравнении с большими, тяжеловестными, например, BERT-like моделями.

##**Воспроизведение:**
Воспроизвести можно, например, в Colab положив рядом файл с датасетом dataset_1937770_3.txt

In [5]:
"""Hybrid Quality Solution for Space Restoration - Combined Approach"""

import pandas as pd
import re
import numpy as np
from collections import Counter
from tqdm.auto import tqdm

class HybridSpaceRestorer:
    """
    Гибридный восстановитель пробелов, объединяющий:
    1. Словарный поиск
    2. Алгоритм Витерби с Avito-словарем
    3. Лингвистические эвристики
    4. Паттерны и правила
    """

    def __init__(self):
        # Загружаем оба словаря
        self.fast_dict = self._load_comprehensive_dictionary()
        self.avito_dict = self._create_avito_corpus([])  # Будет обновлен позже
        self.patterns = self._load_patterns()

        # Параметры для взвешивания методов
        self.dict_weight = 0.6
        self.viterbi_weight = 0.3
        self.heuristic_weight = 0.1

    def _load_comprehensive_dictionary(self):
        """Расширенный словарь русских слов для Avito"""
        words = [
            # Глаголы и действия (самые частые)
            'куплю', 'продам', 'сдам', 'ищу', 'сниму', 'меняю', 'отдам', 'приму',
            'покупаю', 'продаю', 'сдаю', 'ищем', 'снимаем', 'меняем',
            'ремонт', 'услуги', 'заказ', 'доставка', 'продажа', 'покупка', 'обмен',

            # Недвижимость
            'квартира', 'дом', 'комната', 'офис', 'гараж', 'участок', 'дача', 'студия',
            'апартаменты', 'таунхаус', 'коттедж', 'балкон', 'кухня', 'санузел', 'гостиная',
            'спальня', 'ванная', 'коридор', 'кладовая', 'мастерская',

            # Техника и электроника
            'айфон', 'телефон', 'смартфон', 'ноутбук', 'компьютер', 'телевизор',
            'холодильник', 'стиральная', 'посудомойка', 'микроволновка', 'пылесос',
            'кофемашина', 'утюг', 'фен', 'блендер', 'миксер', 'мясорубка', 'чайник',
            'роутер', 'планшет', 'камера', 'наушники', 'колонки', 'монитор',

            # Мебель
            'диван', 'кровать', 'стол', 'стул', 'шкаф', 'комод', 'полка', 'кресло',
            'тумба', 'стеллаж', 'пуф', 'банкетка', 'сервант', 'буфет', 'гарнитур',

            # Транспорт
            'машина', 'авто', 'автомобиль', 'велосипед', 'мотоцикл', 'скутер',
            'самокат', 'гироскутер', 'электросамокат',

            # Прилагательные и характеристики
            'новый', 'новая', 'новое', 'б/у', 'бу', 'срочно', 'недорого', 'дешево',
            'дорого', 'отличный', 'хороший', 'качественный', 'евро', 'оригинал',
            'рабочий', 'исправный', 'чистый', 'аккуратный', 'современный', 'стильный',

            # Местоположение
            'москва', 'питер', 'спб', 'подмосковье', 'область', 'центр', 'метро',
            'мкад', 'садовая', 'невский', 'арбат', 'тверская', 'ленинский', 'проспект',

            # Единицы измерения и цены
            'руб', 'рублей', 'долларов', 'евро', 'шт', 'см', 'м', 'мм', 'кг', 'л', 'кв',
            'цена', 'стоимость', 'бюджет', 'скидка', 'акция', 'распродажа',

            # Профессии и услуги
            'работа', 'вакансия', 'требуется', 'ищем', 'грузчик', 'водитель', 'курьер',
            'разнорабочий', 'строитель', 'отделочник', 'сантехник', 'электрик',
            'мастер', 'специалист', 'опыт', 'образование',

            # Разное
            'кошка', 'собака', 'котенок', 'щенок', 'аквариум', 'клетка', 'переноска',
            'игрушки', 'одежда', 'обувь', 'куртка', 'платье', 'джинсы', 'футболка'
        ]

        return set(words)

    def _create_avito_corpus(self, texts):
        """Создаем Avito-ориентированный словарь (из второго подхода)"""
        avito_categories = {
            'недвижимость': [
                'куплю', 'продам', 'сниму', 'сдам', 'аренда', 'посуточно', 'квартира',
                'комната', 'дом', 'дача', 'гараж', 'офис', 'студия', 'новостройка',
                'вторичка', 'ипотека', 'залог', 'без', 'посредников', 'собственник',
            ],
            'транспорт': [
                'купить', 'продать', 'авто', 'машина', 'автомобиль', 'бушный', 'новый',
                'подержанный', 'пробег', 'год', 'выпуска', 'цвет', 'двигатель',
            ],
            'электроника': [
                'телефон', 'смартфон', 'айфон', 'iphone', 'самсунг', 'samsung', 'xiaomi',
                'ноутбук', 'компьютер', 'пк', 'планшет', 'ipad', 'телевизор',
            ]
        }

        all_words = []
        for category in avito_categories.values():
            all_words.extend(category)

        # Добавляем слова из текстов
        for text in texts:
            words = re.findall(r'[а-яё]{3,12}', text.lower())
            all_words.extend(words)

        word_freq = Counter(all_words)

        # Нормализуем частоты
        if word_freq:
            max_freq = max(word_freq.values())
            for word in word_freq:
                word_freq[word] = word_freq[word] / max_freq * 1000000

        return word_freq

    def _load_patterns(self):
        """Загружаем паттерны для поиска"""
        return {
            'prices': [r'\d+руб', r'\d+р', r'\d+₽', r'\d+\$', r'\d+USD', r'\d+евро'],
            'sizes': [r'\d+см', r'\d+м', r'\d+мм', r'\d+кв', r'\d+л', r'\d+кг'],
            'phones': [r'\d{11}', r'\d{10}', r'\d{7}'],
            'dates': [r'\d{2}\.\d{2}', r'\d{2}\.\d{2}\.\d{4}']
        }

    def fit(self, texts):
        """Обучение на текстах (обновление Avito-словаря)"""
        self.avito_dict = self._create_avito_corpus(texts)
        print(f"Обучен Avito-словарь на {len(texts)} текстах")

    def word_probability_avito(self, word):
        """Вероятность слова в Avito-контексте"""
        word_lower = word.lower()

        if word_lower in self.avito_dict:
            base_prob = np.log(self.avito_dict[word_lower] + 1)

            # Бонусы для Avito-ключевых слов
            avito_keywords = ['куплю', 'продам', 'сдам', 'сниму', 'ищу', 'бу', 'новый', 'срочно']
            if word_lower in avito_keywords:
                base_prob += 3

            return base_prob
        else:
            # Штраф для неизвестных слов
            penalty = -len(word) * 2
            if re.match(r'^[а-яё]+$', word_lower):
                penalty += 1
            if len(word) <= 2:
                penalty -= 15
            return penalty

    def viterbi_search(self, text):
        """Алгоритм Витерби для поиска оптимального разбиения"""
        n = len(text)
        if n < 3:
            return []

        # Инициализация
        dp = [-float('inf')] * (n + 1)
        dp[0] = 0
        prev = [None] * (n + 1)

        max_word_len = 12

        for i in range(1, n + 1):
            for j in range(max(0, i - max_word_len), i):
                if j < 0:
                    continue

                word = text[j:i]
                prob = self.word_probability_avito(word)

                if dp[j] + prob > dp[i]:
                    dp[i] = dp[j] + prob
                    prev[i] = j

        # Восстановление позиций
        spaces = []
        i = n
        while i > 0:
            j = prev[i]
            if j is not None and j > 0:
                spaces.append(j - 1)
            i = j
            if i is None:
                break

        return sorted(spaces)

    def dictionary_search(self, text):
        """Быстрый словарный поиск (из первого подхода)"""
        spaces = []
        text_lower = text.lower()

        # Ищем самые длинные слова сначала
        words_sorted = sorted(self.fast_dict, key=len, reverse=True)

        i = 0
        while i < len(text):
            found = False
            for word in words_sorted:
                if len(word) <= len(text) - i and text_lower[i:i+len(word)] == word:
                    if i + len(word) < len(text):
                        spaces.append(i + len(word))
                    i += len(word)
                    found = True
                    break

            if not found:
                i += 1

        return spaces

    def pattern_search(self, text):
        """Поиск паттернов (цены, размеры и т.д.)"""
        spaces = []

        # Цены и размеры
        for pattern_type in ['prices', 'sizes']:
            for pattern in self.patterns[pattern_type]:
                for match in re.finditer(pattern, text):
                    if match.start() > 0:
                        spaces.append(match.start())

        return spaces

    def russian_heuristics(self, text):
        """Лингвистические эвристики для русского языка"""
        spaces = []

        vowels = 'аеёиоуыэюя'
        consonants = 'бвгджзйклмнпрстфхцчшщ'

        # Эвристика: после предлогов
        short_words = ['в', 'на', 'за', 'под', 'над', 'от', 'до', 'по', 'со', 'из', 'не', 'без']
        for word in short_words:
            start = 0
            while True:
                pos = text.find(word, start)
                if pos == -1:
                    break
                if pos + len(word) < len(text):
                    next_char = text[pos + len(word)]
                    if next_char.lower() in consonants:
                        spaces.append(pos + len(word))
                start = pos + 1

        # Эвристика: границы слов по чередованию гласных-согласных
        for i in range(2, len(text) - 2):
            if (text[i] in consonants and text[i+1] in vowels and
                text[i-1] in vowels and text[i+2] in consonants):
                if i + 1 not in spaces:
                    spaces.append(i + 1)

        return spaces

    def number_search(self, text):
        """Поиск и обработка чисел"""
        spaces = []

        for match in re.finditer(r'\d+', text):
            start, end = match.span()

            if start > 0:
                spaces.append(start)

            if end < len(text):
                next_part = text[end:end+3]
                if not any(pattern in next_part for pattern in ['руб', 'р', 'см', 'м', 'кг', 'л', 'кв']):
                    spaces.append(end)

        return spaces

    def combine_results(self, text, all_spaces):
        """Комбинирование результатов от разных методов"""
        if not all_spaces:
            return []

        # Собираем все предложенные позиции с весами
        space_scores = {}

        # Метод 1: Словарный поиск (высокий вес)
        dict_spaces = set(all_spaces.get('dictionary', []))
        for space in dict_spaces:
            space_scores[space] = space_scores.get(space, 0) + self.dict_weight

        # Метод 2: Витерби (средний вес)
        viterbi_spaces = set(all_spaces.get('viterbi', []))
        for space in viterbi_spaces:
            space_scores[space] = space_scores.get(space, 0) + self.viterbi_weight

        # Метод 3: Эвристики (низкий вес)
        heuristic_spaces = set(all_spaces.get('heuristics', []))
        for space in heuristic_spaces:
            space_scores[space] = space_scores.get(space, 0) + self.heuristic_weight

        # Метод 4: Паттерны (высокий вес для найденных)
        pattern_spaces = set(all_spaces.get('patterns', []))
        for space in pattern_spaces:
            space_scores[space] = space_scores.get(space, 0) + 0.8  # Высокий вес

        # Отбираем позиции с достаточным весом
        threshold = 0.3
        candidate_spaces = [space for space, score in space_scores.items()
                          if score >= threshold]

        return self._filter_spaces(text, candidate_spaces)

    def _filter_spaces(self, text, spaces):
        """Фильтрация пробелов"""
        if not spaces:
            return []

        unique_spaces = sorted(set(spaces))
        filtered = []
        prev_space = -3

        for space in unique_spaces:
            # Проверяем минимальное расстояние
            if space - prev_space >= 2 and 1 <= space < len(text):
                # Проверяем, что не разбиваем числа или английские слова
                if not (space > 0 and text[space-1].isdigit() and
                       space < len(text) and text[space].isdigit()):
                    if not (text[space-1].isalpha() and text[space].isalpha() and
                           text[space-1].isascii() and text[space].isascii()):
                        filtered.append(space)
                        prev_space = space

        return filtered

    def restore_spaces(self, text):
        """Основной метод восстановления пробелов"""
        if not text or len(text) < 3:
            return []

        # Очищаем текст
        clean_text = self._clean_text(text)
        if len(clean_text) < 3:
            return []

        # Применяем все методы
        all_spaces = {}

        # 1. Быстрый словарный поиск
        all_spaces['dictionary'] = self.dictionary_search(clean_text)

        # 2. Алгоритм Витерби
        all_spaces['viterbi'] = self.viterbi_search(clean_text)

        # 3. Лингвистические эвристики
        all_spaces['heuristics'] = self.russian_heuristics(clean_text)

        # 4. Поиск паттернов
        all_spaces['patterns'] = self.pattern_search(clean_text)

        # 5. Поиск чисел
        all_spaces['numbers'] = self.number_search(clean_text)

        # Комбинируем результаты
        final_spaces = self.combine_results(clean_text, all_spaces)

        return final_spaces

    def _clean_text(self, text):
        """Очищаем текст от ID"""
        if re.match(r'^\d+,', text):
            return text.split(',', 1)[1]
        return text

def load_data(filename):
    """Загружаем данные с обработкой запятых"""
    data = []
    with open(filename, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[1:]:
            parts = line.strip().split(',', 1)
            if len(parts) == 2:
                data.append(parts)
            else:
                data.append([parts[0], ' '.join(parts[1:])])

    df = pd.DataFrame(data, columns=['id', 'text_no_spaces'])
    df['id'] = df['id'].astype(int)
    return df

def create_hybrid_submission():
    """Создаем гибридный submission"""

    print("Creating HYBRID space restoration solution...")
    print("=" * 60)

    # Загружаем данные
    filename = 'dataset_1937770_3.txt'
    data = load_data(filename)
    texts = data['text_no_spaces'].tolist()

    print(f"Loaded {len(texts)} texts")

    # Инициализируем гибридный восстановитель
    restorer = HybridSpaceRestorer()

    # Обучаем на данных
    print("Training hybrid model...")
    restorer.fit(texts)

    # Генерируем предсказания
    print("Generating hybrid predictions...")
    predictions = []

    for i, text in tqdm(enumerate(texts), total=len(texts)):
        spaces = restorer.restore_spaces(text)
        predictions.append(spaces)

    # Создаем результат
    data['predicted_positions'] = predictions
    data['predicted_positions'] = data['predicted_positions'].apply(
        lambda x: str(x) if x else '[]'
    )

    # Сохраняем
    output_file = 'hybrid_submission.csv'
    data[['id', 'predicted_positions']].to_csv(output_file, index=False)

    print(f"Saved to {output_file}")

    # Тестируем на примерах
    print("\nTesting hybrid model on examples:")
    print("=" * 60)

    test_cases = [
        "куплюайфон14про",
        "сдаюквартирувцентреМосквы",
        "ищуработуудаленновМоскве",
        "новыйдивансдоставкойнедорого",
        "отдамкошкувдобрыерукисрочно"
    ]

    for text in test_cases:
        spaces = restorer.restore_spaces(text)

        # Восстанавливаем текст для демонстрации
        if spaces:
            result = list(text)
            for pos in sorted(spaces, reverse=True):
                if pos < len(result):
                    result.insert(pos + 1, ' ')
            restored = ''.join(result)
        else:
            restored = text

        print(f"Input:  {text}")
        print(f"Output: {restored}")
        print(f"Spaces: {spaces}")
        print("-" * 50)

    # Анализ эффективности методов
    print("\nMethod effectiveness analysis:")
    print("=" * 60)

    sample_text = "куплюайфон14просрочнонемедорого"
    methods = ['dictionary', 'viterbi', 'heuristics', 'patterns']

    all_method_spaces = {}
    for method in methods:
        if method == 'dictionary':
            spaces = restorer.dictionary_search(sample_text)
        elif method == 'viterbi':
            spaces = restorer.viterbi_search(sample_text)
        elif method == 'heuristics':
            spaces = restorer.russian_heuristics(sample_text)
        elif method == 'patterns':
            spaces = restorer.pattern_search(sample_text)

        all_method_spaces[method] = spaces

        # Демонстрация результата метода
        if spaces:
            result = list(sample_text)
            for pos in sorted(spaces, reverse=True):
                if pos < len(result):
                    result.insert(pos + 1, ' ')
            method_result = ''.join(result)
        else:
            method_result = sample_text

        print(f"{method:12}: {method_result}")
        print(f"{' ':12}  Spaces: {spaces}")

    # Комбинированный результат
    final_spaces = restorer.combine_results(sample_text, all_method_spaces)
    if final_spaces:
        result = list(sample_text)
        for pos in sorted(final_spaces, reverse=True):
            if pos < len(result):
                result.insert(pos + 1, ' ')
        final_result = ''.join(result)
    else:
        final_result = sample_text

    print(f"{'HYBRID':12}: {final_result}")
    print(f"{' ':12}  Spaces: {final_spaces}")

    return data

if __name__ == "__main__":
    result_df = create_hybrid_submission()

    print("\nHybrid solution completed!")
    print("Combined strengths of both approaches:")
    print("   • Fast dictionary search")
    print("   • Viterbi algorithm with Avito corpus")
    print("   • Linguistic heuristics")
    print("   • Pattern recognition")
    print("   • Intelligent result combination")
    print("Expected: Maximum F1 score improvement!")

Creating HYBRID space restoration solution...
Loaded 1005 texts
Training hybrid model...
Обучен Avito-словарь на 1005 текстах
Generating hybrid predictions...


  0%|          | 0/1005 [00:00<?, ?it/s]

Saved to hybrid_submission.csv

Testing hybrid model on examples:
Input:  куплюайфон14про
Output: куплю айфон 14про
Spaces: [4, 9]
--------------------------------------------------
Input:  сдаюквартирувцентреМосквы
Output: сдаюк ва ртиру вцентре Мо сквы 
Spaces: [4, 6, 11, 18, 20, 24]
--------------------------------------------------
Input:  ищуработуудаленновМоскве
Output: ищур абот ууд але нно вМо скве 
Spaces: [3, 7, 10, 13, 16, 19, 23]
--------------------------------------------------
Input:  новыйдивансдоставкойнедорого
Output: новыйд иван сдо ста вкой недорого
Spaces: [5, 9, 12, 15, 19]
--------------------------------------------------
Input:  отдамкошкувдобрыерукисрочно
Output: отд амк ошку вдо бры еруки срочно
Spaces: [2, 5, 9, 12, 15, 20]
--------------------------------------------------

Method effectiveness analysis:
dictionary  : куплюа йфон1 4просрочнон еме дорого
              Spaces: [5, 10, 21, 24]
viterbi     : куплю айфон 14 про срочно нем едорого
              S